In [38]:
import pandas as pd
import numpy as np
import microdf as mdf

p = pd.read_csv("../data/asec_2019_ipums.csv.gz")

# Preprocess
p.columns = p.columns.str.lower()

# Replace NIU codes.
NIU_CODES = {
    "adjginc": 99999999,
    "offtotval": 9999999999,
    # Secondary individuals under 15 have NIU for cutoff.
    # We set them to zero as to give them no UBI
    # (they're still part of SPM units with others who get UBI).
    "cutoff": 999999,
}
for column, niu in NIU_CODES.items():
    p[column].replace({niu: 0}, inplace=True)

p["fam"] = p.marbasecidh.astype(str) + "-" + p.famid.astype(str)

# Create aggregates.

# Family: note families are assigned only to one SPM unit.
f = p.groupby(["fam", "cutoff", "offtotval", "spmfamunit"]).adjginc.sum().reset_index().rename(columns={"adjginc": "fam_adjginc"})

# SPM unit, with all SPM characteristics.
s = p[['spmlunch', 'spmcaphous', 'spmwt', 'spmeitc',
       'spmwic', 'spmheat', 'spmsnap', 'spmtotres', 'spmthresh', 'spmfamunit']].drop_duplicates()

In [39]:
s

,spmlunch,spmcaphous,spmwt,spmeitc,spmwic,spmheat,spmsnap,spmtotres,spmthresh,spmfamunit
0,0,0.0,1552.90,0,0.0,0.0,0,86459.0,14700.0,1001
2,0,0.0,990.49,0,0.0,0.0,0,55275.0,14700.0,2001
4,0,0.0,1505.27,0,0.0,0.0,0,28653.0,11920.0,3001
5,0,0.0,1430.70,529,0.0,0.0,720,6989.0,16630.0,4001
7,0,0.0,1133.37,0,0.0,0.0,0,44730.0,27520.0,5001
...,...,...,...,...,...,...,...,...,...,...
157953,0,0.0,486.02,0,0.0,0.0,0,17428.0,14770.0,91497001
157954,0,0.0,444.18,0,0.0,0.0,0,42819.0,14770.0,91498001
157955,0,0.0,388.29,244,0.0,0.0,0,-3574.0,14770.0,91498003
157956,0,7042.0,494.05,0,0.0,200.0,0,20868.0,17800.0,91499001


In [2]:
p.columns

Index(['year', 'serial', 'month', 'cpsid', 'asecflag', 'asecwth',
       'marbasecidh', 'pernum', 'cpsidp', 'asecwt', 'age', 'race',
       'marbasecidp', 'famid', 'inctot', 'incwelfr', 'adjginc', 'offtotval',
       'offcutoff', 'cutoff', 'spmlunch', 'spmcaphous', 'spmwt', 'spmeitc',
       'spmwic', 'spmheat', 'spmsnap', 'spmtotres', 'spmthresh', 'spmfamunit',
       'fam'],
      dtype='object')

In [115]:
def phase_out(amount, rate, respect_to):
    return np.maximum(0, amount - respect_to * rate)


def ubi_fpg(phase_out_rate=0, respect_to="offtotval", repeal_bens=None):
    f["chg"] = phase_out(f.cutoff, phase_out_rate, np.maximum(0, f[respect_to]))
    s2 = s.merge(f.groupby("spmfamunit").chg.sum().reset_index(), on="spmfamunit")
    if repeal_bens is not None and repeal_bens != [""]:
        s2.chg -= s2[repeal_bens].sum(axis=1)
    s2.spmtotres += s2.chg
    p2 = p.drop(columns="spmtotres").merge(s2[["spmtotres", "spmfamunit"]], on="spmfamunit")
    pov = mdf.poverty_rate(p2, "spmtotres", "spmthresh", "asecwt")
    cost = mdf.weighted_sum(s2, "chg", "spmwt")
    return pd.Series({"poverty": pov, "cost": cost})

In [116]:

sim = mdf.cartesian_product({"phase_out": [np.inf, 0.5, 0.25, 0], "repeal_bens": ["", "spmlunch,spmsnap,spmeitc"]})
sim = pd.concat([sim, sim.apply(lambda x: ubi_fpg(x.phase_out, repeal_bens=x.repeal_bens.split(",")), axis=1)], axis=1)
sim
# pd.Series(PHASE_OUTS, index=PHASE_OUTS).apply(ubi_fpg).reset_index()
# ubi_fpg(np.inf)

,phase_out,repeal_bens,poverty,cost
0,inf,,0.117132,0.000000e+00
1,inf,"spmlunch,spmsnap,spmeitc",0.143566,-7.752553e+10
2,0.50,,0.031919,3.442652e+11
3,0.50,"spmlunch,spmsnap,spmeitc",0.040664,2.667397e+11
4,0.25,,0.018187,7.552311e+11
5,0.25,"spmlunch,spmsnap,spmeitc",0.021952,6.777056e+11
6,0.00,,0.013046,2.708504e+12
7,0.00,"spmlunch,spmsnap,spmeitc",0.015262,2.630979e+12


In [55]:
c.groupby("fam").spmfamunit.nunique().max()

1

In [57]:
c.groupby("spmfamunit").fam.nunique().max()

8

In [29]:
c.columns

Index(['year', 'serial', 'month', 'cpsid', 'asecflag', 'asecwth', 'pernum',
       'cpsidp', 'asecwt', 'age', 'race', 'famid', 'inctot', 'incwelfr',
       'adjginc', 'offtotval', 'offcutoff', 'cutoff', 'spmlunch', 'spmcaphous',
       'spmwt', 'spmeitc', 'spmwic', 'spmheat', 'spmsnap', 'spmtotres',
       'spmthresh', 'spmfamunit'],
      dtype='object')

In [ ]:
pd.read_csv()

In [60]:
c[c.cpsid == 20200307030400][["fam", "offcutoff", "offtotval"]]

,fam,offcutoff,offtotval
118006,1120175155-1,17622,45015
118007,1120175155-1,17622,45015
118008,1120175155-2,13300,35007


* Aggregate to family and SPM unit
* Determine 

In [22]:
c.groupby("cpsidp").size().max()

53081

In [7]:
c.groupby("offcutoff").size().sort_index()

offcutoff
12261      7520
13300     18105
15453     13728
17120     17512
17555       140
17622      4172
19998     10284
20578     13035
20598      4389
25926     20400
26017      2336
26370      4816
26801      6720
30044       840
30510     10030
31275      4465
31800      1415
32263      2645
33522       294
34161      3756
35239      1968
35965      1878
36576       312
36721       798
36757        91
38262      1092
39635       707
40811       735
41442       483
41709        56
42066       384
42085        84
42348       245
43470       232
44818       304
45881       200
46630       176
47069        24
47485        56
49426       162
51406       245
51727       189
53025       252
54460       163
55503        93
56139        75
56621        19
56895        37
999999      297
dtype: int64

In [9]:
c.groupby("cutoff").size().sort_index()

cutoff
12261      7520
13300     18105
15453     13728
17120     17512
17555       140
17622      4172
19998     10284
20578     13035
20598      4389
25926     20400
26017      2336
26370      4816
26801      6720
30044       840
30510     10030
31275      4465
31800      1415
32263      2645
33522       294
34161      3756
35239      1968
35965      1878
36576       312
36721       798
36757        91
38262      1092
39635       707
40811       735
41442       483
41709        56
42066       384
42085        84
42348       245
43470       232
44818       304
45881       200
46630       176
47069        24
47485        56
49426       162
51406       245
51727       189
53025       252
54460       163
55503        93
56139        75
56621        19
56895        37
999999      297
dtype: int64